The following exercises are meant to be solved by gathering the bash commands incrimentally in two scripts, one for ex 1.* the other for ex 2.* 

### Ex 1

1\.a Make a new directory called `students` in your home. Download a csv file with the list of students of this lab from [here](https://www.dropbox.com/s/867rtx3az6e9gm8/LCP_22-23_students.csv) (use the `wget` command) and copy that to `students`. First check whether the file is already there

```bash

#Going home
cd $HOME

#Creating students directory
mkdir students

#Downloading the file
wget https://www.dropbox.com/s/867rtx3az6e9gm8/LCP_22-23_students.csv

#Copying the file into the students directory
cp LCP_22-23_students.csv ./students

#Checking if the file is there
find ./students -name "LCP_22-23_students.csv"

1\.b Make two new files, one containing the students belonging to PoD, the other to Physics.

```bash


#Creating the PoD file
grep 'PoD' LCP_22-23_students.csv > PoD_students.csv

#Creating the Physics file
grep 'Physics' LCP_22-23_students.csv > Physics_students.csv

1\.c For each letter of the alphabet, count the number of students whose surname starts with that letter. 

1\.d Find out which is the letter with most counts.

```bash

#!/bin/bash

max_count=0
max_letter=''

for letter in {A..Z}; do

    count=$(tail -n +2 LCP_22-23_students.csv | grep -c "^$letter")   #The instruction 'tail -n +(N+1)' skips the first N rows
    echo "$letter: $count"

    if [ $count -gt $max_count ]; then
        max_count=$count
        max_letter=$letter
    fi

done

echo "The most frequent letter is $max_letter counted $max_count times"

1\.e Assume an obvious numbering of the students in the file (first line is 1, second line is 2, etc.), group students "modulo 18", i.e. 1,19,37,.. 2,20,38,.. etc. and put each group in a separate file  

```bash

#!/bin/bash

modulo=18    #Set the modulo

total=$(grep -c -v "^#" LCP_22-23_students.csv)   #Counting the rows of the file

i=2   #Inizializing the parameter to 2 because we wanna skip the first row and start from the second one

while [ $i -lt $(($modulo+2)) ]; do    #We want to get from 1 to 18, so if we start from i=2 we have to get to i=19, hence the condition is i<20

        file_name="students_$(($i-1)).csv"  #Naming the file where we will save the students
        j=$i                                #Fixing j=i because the first number in every sequence follows the one from the previous sequence (*1*,19,... and *2*,20,... and *3*,21,... ecc.)

        while [ $j -le $total ]; do    #Now we go into every sequence

                #sed -n "${j}p" copies the j-th row from the actual file to the new one; also >> appends a new row to the existing ones
                sed -n "${j}p" LCP_22-23_students.csv >> "$file_name"
                j=$(($j+$modulo))   #Updating the counting of j

        done

        i=$(($i+1))   #Updating the counter i

done

### Ex 2

2.a Make a copy of the file `data.csv` removing the metadata and the commas between numbers; call it `data.txt`

```bash

#Instruction sed 's/a/b/g' substitues character 'a' with character 'b' in every row (the final 'g' means 'global')
grep -v '^#' data.csv | sed 's/,//g' > data.txt

2\.b How many even numbers are there?

```bash

#!/bin/bash

count=0

for (( i=1; i<=$(grep -c "" data.txt); i++ )); do   #I iterate through all the rows of the file
    for num in $(sed -n "${i}p" data.txt); do       #With sed -n "${i}p" we work with the i-th row like it is a vector

        if [ $(($num % 2)) -eq 0 ]; then            #Checking if it is even by dividing by two and getting the rest
                count=$(($count+1));                #Uploading the count
        fi

    done
done

echo "Number of even numbers: $count"

2\.c Distinguish the entries on the basis of `sqrt(X^2 + Y^2 + Z^2)` is greater or smaller than `100*sqrt(3)/2`. Count the entries of each of the two groups 

I've interpreted the 'entires' as the entire row of six numbers. I'm still in doubt as to whether it's meant as the triplet of (X,Y,Z) or (X',Y',Z'), and in this case each rows would have two entires.

```bash

#!/bin/bash

bound=$( echo 'scale=6; 100*sqrt(3)/2' | bc)
count_smaller=0
count_greater=0

for (( i=1; i<=$(grep -c "" data.txt); i++ )); do    #Scrolls on every row of file

        square=0              #Inizializing the sum at zero
        only_first_three=1    #Parameter we need to sum just the first three numbers of every row (which are X, Y and Z)

        for num in $(sed -n "${i}p" data.txt); do   #Scrolls on every number of the row

                if [ $only_first_three -gt 3 ]; then  #If we go beyond the first three numbers...
                        break                         #...we break the cycle
                fi

                square=$(( $square + ( $num * $num )))       #Uploading the sum (X^2+Y^2+Z^2)
                only_first_three=$(($only_first_three + 1))  #Uploading the first-three-numbers parameter

        done

        square=$( echo "scale=6; sqrt($square)" | bc)    #Getting the square root of X^2+Y^2+Z^2

        #Counting the smaller group
        if (( $(echo "$square < $bound" | bc) )); then
                count_smaller=$(($count_smaller + 1))

        #Counting the greater group
        elif (( $(echo "$square > $bound" | bc) )); then
                count_greater=$(($count_greater + 1))

        fi

done

echo "There are $count_smaller entries smaller than 100*sqrt(3)/2"
echo "There are $count_greater entries greater than 100*sqrt(3)/2"

2\.d Make `n` copies of data.txt (with `n` an input parameter of the script), where the i-th copy has all the numbers divided by i (with `1<=i<=n`).

```bash

#!/bin/bash

name_file="data.txt"

#Asking how much copies to the user
echo "How many copies of $name_file do you want?"
read n

for (( i=1; i<=n; i++ )); do

        new_file="data$i.txt"  #Naming the files

        for (( j=1; j<=$(grep -c "" data.txt); j++ )); do

                for num in $(sed -n "${j}p" data.txt); do       #For every number of the row
                        num=$(echo "scale=6; $num / $i" | bc)   #Divide it by i
                        echo -n "$num " >> "$new_file"          #Print it in the file without going in new line (-n)
                done

                echo >> "$new_file"   #Print newline

        done

done